In [1]:
import os, sys
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from ipywidgets import interact, Button
import ipywidgets as widgets
import scipy.ndimage as ndimage
from scipy.optimize import curve_fit
from matplotlib.patches import Circle
import xarray
import scipy as sp

In [2]:
sys.path.insert(1, os.path.join(os.path.join(os.path.abspath(""), os.pardir), os.pardir))
import neurotorch as neuro
neuro.Start_Background()
%matplotlib inline

In [ ]:
frames = neuro.API.Peaks()
print(frames)
synapses = neuro.API.DetectedSynapses()
for s in synapses:
    print(s.GetUniqueName())

In [ ]:
# Synapse 4 (237,135)
loc = synapses[1].location
bbox = (loc[0]-30, loc[1]-25, loc[0]+30, loc[1]+25) # (X0, Y0, X1, Y1)
plt.imshow(neuro.API_IMG().imgDiffMaxTime[bbox[1]:bbox[3],bbox[0]:bbox[2]])
plt.show()

_diffMax = np.max(neuro.API_IMG().imgDiff)

def gaus(x,a,x0,sigma):
    return a*np.exp(-(x-x0)**2/(2*sigma**2))

def Plot(peak):
    global _diffMax, frames, xsclice, ysclice
    fig = plt.figure(figsize=(13,5))
    ax1 = fig.add_subplot(131)
    ax2 = fig.add_subplot(132)
    ax3 = fig.add_subplot(133)
    img = neuro.API_IMG().imgDiff[frames[peak], bbox[1]:bbox[3], bbox[0]:bbox[2]]
    xsclice = np.max(img, axis=0)
    xsclice_X = range(len(xsclice))
    ysclice = np.max(img, axis=1)
    ysclice_X = range(len(ysclice))
    ax1.imshow(img, vmin=0, vmax=_diffMax)
    ax2.plot(xsclice)
    ax3.plot(ysclice)

    try:
        poptX,pcovX = curve_fit(gaus, xsclice_X, xsclice, p0=(1,30,1))
        pstdX = np.sqrt(np.diag(pcovX))
        fitXSlice_Y = gaus(xsclice_X, *poptX)
        ax2.set_title(f"X Slice\nx= {round(poptX[1],2)} ± {round(pstdX[1],2)}, σ = {round(poptX[2],2)} ± {round(pstdX[2],2)}")
        ax2.plot(fitXSlice_Y)
    except RuntimeError:
        print("X Fit not possible")
    
    try:
        poptY,pcovY = curve_fit(gaus, ysclice_X, ysclice, p0=(1,30,1))
        pstdY = np.sqrt(np.diag(pcovY))
        fitYSlice_Y = gaus(ysclice_X, *poptY)
        ax3.set_title(f"Y Slice\ny= {round(poptY[1],2)} ± {round(pstdY[1],2)}, σ = {round(poptY[2],2)} ± {round(pstdY[2],2)}")
        ax3.plot(fitYSlice_Y)
    except RuntimeError:
        print("Y Fit not possible")

    if 'poptY' in locals() and 'poptX' in locals():
        gaussCenter = (poptX[1], poptY[1])
        c = Circle(gaussCenter, 1, color="red", fill=False)
        ax1.add_patch(c)
        
    plt.show()

interact(Plot, peak=widgets.IntSlider(min=0, max=len(frames)-1))


In [ ]:
mask = np.array([[[0]],[[0]],[[1]],[[1]], [[1]]])
print(mask[:,0,0], mask.shape)
from scipy.signal import convolve
imgDiffConv = convolve(neuro.API_IMG().imgDiff, mask, mode="same")
print(imgDiffConv.shape)

In [ ]:
gaussImg = sp.ndimage.gaussian_filter(neuro.API_IMG().imgDiff, sigma=2, axes=(1,2))
print(gaussImg.shape)

In [29]:
neuro.API_IMG().CalcDiff()
neuro.API_IMG().CalcDiffMax()
neuro.API_GUI().NewImageProvided()
oldDiff = neuro.API_IMG().imgDiff

In [7]:
neuro.API_IMG().imgDiff = imgDiffConv
neuro.API_IMG().CalcDiffMax()
neuro.API_GUI().NewImageProvided()

In [ ]:
plt.imshow(imgDiffConv[100,:,:])
plt.show()

In [ ]:
plt.imshow(neuro.API_IMG().imgDiff[121,:,:])
plt.show()

In [ ]:
plt.plot(imgDiffConv[70:130,135,230])
plt.plot(oldDiff[70:130,135,230], c="orange")
plt.show()

In [ ]:
mask = np.array([[[0]],[[0]],[[1]],[[1]], [[1]]])
testImg = np.array([[[1,1],[0,1]], [[2,1],[0,1]], [[3,1],[0,1]], [[4,10],[0,1]], [[5,1],[0,1]], [[6,0],[0,1]], [[7,0],[0,1]], [[8,0],[0,1]]])
print(mask[:,0,0], mask.shape, testImg.shape)
imgDiffConvTest = convolve(testImg, mask, mode="same")

In [ ]:
plt.plot(testImg[:,0,0])
plt.plot(imgDiffConvTest[:,0,0], c="orange")
plt.show()
print(imgDiffConvTest[4,0,0])

In [ ]:
plt.plot(testImg[:,0,0])
plt.plot(imgDiffConvTest[:,0,0], c="orange")
plt.show()
print(imgDiffConvTest[4,0,0])

In [ ]:
gaussImg = sp.ndimage.gaussian_filter(neuro.API_IMG().imgDiff, sigma=2, axes=(1,2))
imgDiffGaussMaxTime = np.max(gaussImg, axis=0)
plt.imshow(imgDiffGaussMaxTime)
plt.show()

In [37]:
mask = np.array([[[0]],[[0]],[[1]],[[1]], [[1]]])
imgDiffConv = convolve(gaussImg, mask, mode="same")

In [ ]:
imgDiffGaussConvMaxTime = np.max(imgDiffConv, axis=0)

plt.imshow(imgDiffGaussConvMaxTime)
plt.show()

In [3]:
imgDiff = sp.ndimage.gaussian_filter(neuro.API_IMG().imgDiff, sigma=2, axes=(1,2))
imgDiff_Stats = {"ClipMin": max(0, np.min(imgDiff)), "Max": np.max(imgDiff)}

In [4]:
neuro.API_IMG().imgDiff = imgDiff
neuro.API_IMG().imgDiffStats = imgDiff_Stats

In [5]:
neuro.API_IMG().CalcDiffMax()

In [6]:
neuro.API_GUI().NewImageProvided()